In [14]:
# 파이썬 프로그램 자동채점 AutoGrader
# Copyrighted by Young Pyo Jun, ypjun@yonsei.ac.kr
# V1.0 (2021.12.19)

# -*- coding:utf-8 -*- 
import sys 
import pandas as pd
import os
import time as t
import tkinter as tk
from tkinter import filedialog
import subprocess as sp
import difflib as dl
import re
import pyautogui as pg

from colorama import init, Fore, Back, Style

SUCCESS = 0
INFINITE_LOOP = 2
EXE_ERROR = 1

#init(autoreset=True)
USE_COLOR = True
if USE_COLOR:
    black = Fore.BLACK
    red = Fore.RED
    green = Fore.GREEN
    blue = Fore.BLUE
    rgreen = Fore.GREEN + Back.RED
    back = Fore.RESET
else:
    black=red=green=blue=rgreen=back=''

ansFile = "ans.py"
solFile = "sol.py"
inFile = "in.txt"

VLINE = 60

# for debugging
rLevel = 0   # 0: 전체자동화, 1: 학생별, 2: 문제별, 3: 보기별
msgBuffer = []
DEBUG = False
BUFF = True
def dprint(*args, log = BUFF, scr = DEBUG, color=black, end='\n'):
    pmsg = ''
    for ar in args:
        pmsg += "{} ".format(ar)
    if log:
        msgBuffer.append(pmsg)
    if scr:
        print(color+pmsg+back, end=end)

# 채점 프로그램이 있는 폴더를 선택
# 폴더구조: 응시자, 채점결과(100점이 아닌 경우)
def SelectFile():
    fname = ""
    window = tk.Tk()
    window.geometry("0x0")
    window.tkraise()
    window.file = filedialog.askdirectory(initialdir=".", title="Select")
    if window.file != None:
        fname = window.file
    window.destroy()
    return fname

# 파일을 복사 (실행할 프로그램을 현재 폴더로 복사, solution 프로그램은 현재 폴더에 이미 있음)
def myCopy(s, d, src="file", type="python", mylib=[]):
    if src == "file":
        fIn = open(s, "rt", encoding='utf-8')
        ct = fIn.read()
        fIn.close()
    else:
        ct = s
    fct = ct
    for lib in mylib:
        myl = "my" + lib
        fct = fct.replace(lib, myl)
    if type == "python":
        fOut = open(d, "wt", encoding='utf-8')
        myInput = "_input=input\ndef input(*ps):\n    if ps: ps = ps[0]\n    else: ps = ''\n    r = _input(ps)\n    print('<입력값>')\n    return r\n"
        fOut.write(myInput)
    else:  # type == "text"
        fOut = open(d, "wt", encoding='euc-kr')
    fOut.write(fct)
    fOut.close()
    return ct

# 주석 제거
def deComment(pgm):
    skip2NL = False
    newPgm = ''
    for ch in pgm:
        if ch == '#':
            skip2NL = True
        if skip2NL:
            if ch == '\n':
                newPgm += ch
                skip2NL = False
            continue
        newPgm += ch
    return newPgm

# 줄 번호 삽임
def numbering(Pgm, tag):
    nPgm = Pgm.splitlines()
    for n in range(len(nPgm)):
        nPgm[n] = "{}{:03d} ||{}".format(tag, n+1, nPgm[n])
    return '\n'.join(nPgm)

def GetPNum(pName):
    if pName.split('.')[-1] != 'py': return 0
    nPS = pName.find('(')
    nPE = pName.find(')')
    if nPS == -1 or nPE == -1:
        n = 0
        while n == 0:
            n = input("Enter number({}): ".format(pName))
            if n.isdigit(): n = int(n)
            else: n = 0
    else:
        st = pName[nPS+1:nPE]
        if st.isdigit():
            n = int(st)
        else:
            n = 0
            while n == 0:
                n = input("Enter number({}): ".format(pName))
                if n.isdigit(): n = int(n)
                else: n = 0
    return n  

#def renewPython(pFile, inData):

def execPython(pFile, inData):
    #inData += "\n\n"
    ret = 0
    out = ''
    err = ''
    exeP = "C:/Users/YPJ/AppData/Local/Programs/Python/Python39/python.exe"
    exeP = "python.exe"
    #rx = sp.Popen(args=[sys.executable, pFile], stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.PIPE, text=True)
    rx = sp.Popen([exeP, pFile], stdin=sp.PIPE, stdout=sp.PIPE, stderr=sp.PIPE, text=True)
    try:  # normal err = 0
        out, err = rx.communicate(input=inData, timeout=5)
        ret = rx.returncode    # SUCCESS (0) or ERROR (1)
    except sp.TimeoutExpired: # 무한반복 발생
        rxpid = rx.pid
        rx.kill()
        sp.run("taskkill /PID {} /T".format(rxpid))
        out, err = rx.communicate()
        if out:
            outLen = len(out)
            out = out[:min(500, len(out))]
        ret = INFINITE_LOOP   # infinite loop (2)
    except Exception: # 실행오류 발생
        ret = EXE_ERROR
    if err:
        ers = err.split('\n')
        if len(ers) > 3:
            err = "{}\t# {}".format(ers[2], ers[3])
    if not out: out = ''
    out = out.rstrip()
    return (ret, out, err)

def ml_rstrip(s):
    slist = [x.rstrip() for x in s.split('\n')]
    return '\n'.join(slist)

def SimRate(src, dst, unit="byte"):  # 문자열 src가 dst에 얼마나 유사한가?
    if not src: return 100
    if not dst: return 0
    sim = 0
    if unit == "word":
        src = src.replace('.','')
        dst = dst.replace('.','')
        src = src.split()   # remove white space
        srcLen = len(src)
        dst = dst.split()
        dstLen = len(dst)    
    elif unit == "byte" or unit == "user":
        src = list(src)   # All characters
        srcLen = len(src)
        dst = list(dst)
        dstLen = len(dst)
    else:  # unit == "char" or unit == "serial":
        src = list(src.replace(' ', ''))    #src = list(''.join(src.split(' ')))   # remove white space
        srcLen = len(src)
        dst = list(dst.replace(' ', ''))    #dst = list(''.join(dst.split(' ')))
        dstLen = len(dst)
    for sc in src:    # src의 요소값이 dst에 얼많큼 있는가?
        if not dst: dst = []; break
        if sc in dst:
            sim += 1
            dst.remove(sc)
    return max(0, (sim * 100 / srcLen) - (len(dst) * 100 / dstLen))

#################################################################################################
# 정보파일
cdir = os.getcwd()
attendee_file = cdir + '\\roll-addrs.xls'  # 학생 정보 파일
scrFile = "ScoringTable.xlsx"              # 채점 정보파일
PLAGIFILE = "plagiCheck.csv"               # 표절자 정보 파일
PlagiCols = ('stid', 'pnum', 'name', 'program', 'who')
cSTID, cPNUM, cNAME, cPROGRAM, cWHO = PlagiCols
bCheckPlagi = False

# 난이도 점수 파일
avgScrFile = "avgscr.txt"
bHaveAvg = False
MINSCORE = 10

HomeDir = SelectFile()    # 분반 폴더 선택

if HomeDir:
    os.chdir(HomeDir)

if os.access(scrFile, os.R_OK):
    hw_name = HomeDir.split('/')[-1]
    stdDir = HomeDir + '/응시자'
    # 감점요율 상수
    rsBanned = 0.5    # 금지어 사용
    rsLoop = 0.9      # 무한반복 발생
    rsErr = 0.9       # 실행오류 발생

    # 점수비율
    srANS = 70
    srSIM = 30
    srNL = 1.0

    # 채점정보 Frame: 번호 / wpahr / 정답프로그램 / 배점 / 필수배제 / Ex 예제들
    scrFile = "ScoringTable.xlsx"
    scrFrame = pd.read_excel(scrFile, dtype=str)
    scrFactor = tuple(scrFrame.배점.fillna(0).astype(int) / 100)
    if round(sum(scrFactor), 3) != 1.0:
        print("WARNING: 점수 배점이 100점이 아닙니다({:.0f}점). 동일배점으로 조정".format(round(sum(scrFactor), 3) * 100))
        scrFactor = tuple(1/len(scrFactor) for n in scrFactor)

    stFolders = os.listdir(stdDir)
    skipName = ""
    pfName = hw_name
    fdBegin = pfName.find('(')
    fdEnd = pfName.find(')')
    if fdBegin != -1 and fdEnd != -1:
        delayTime = pfName[fdBegin+1:fdEnd]
        dt = pd.to_datetime(delayTime)
        dTime = t.mktime((dt.year, dt.month, dt.day, dt.hour, dt.minute, 0, 1, 60, 0))
        hw_name = pfName[:fdBegin].strip()
    else: dTime = t.time()
    # Check for avg file if exist
    if os.access(avgScrFile, os.R_OK):
        bHaveAvg = True
        with open(avgScrFile, "rt") as fp:
            cts = fp.read().split('\n')   # 번호별 평균점수 and 개별 총점 점수
            ct = cts[0]
            if len(cts) > 1: scrScores = list(float(n) for n in cts[1].split())
            else: scrScores = list(0 for n in range(len(stFolders)))
            scrAvg = list(float(n) for n in ct.split())
    else:
        bHaveAvg = False
        scrSum = list(0 for n in range(len(scrFactor)))
        scrAvg = list(0 for n in range(len(scrFactor)))
        scrScores = []
        # hw_name: class_name-class_no 과제제목...
#        if "-" in hw_name:
#            class_name, class_no = hw_name.split('-')
#            class_name = class_name.split()[0]
#            class_no = class_no.split()[0]
#            # 수강생 수 산출
#            gdf = pd.read_excel(attendee_file, dtype=str)
#            gdf = gdf[gdf.교과목명 == class_name]
#            gdf = gdf[gdf.분반 == class_no]
#            scrNumSt = len(gdf)
#        else: scrNumSt = len(stFolders)
    scrNumSt = len(stFolders)
    dprint(f"자동채점({hw_name}) 실행 모드", log=False, scr=True, color=green)
    dprint("  0 : 전체 자동화 (전체 학생에 대하여 한번에 채점)", log=False, scr=True, color=green)
    dprint("  1 : 학생별 자동화 (학생별로 구분하여 채점)", log=False, scr=True, color=green)
    dprint("  2 : 문제별 자동화 (학생에 대해 문제별로 구분하여 채점)", log=False, scr=True, color=green)
    dprint("  3 : 예시별 자동화 (각 문제에 대한 테스트 입력별로 구분하여 채점)", log=False, scr=True, color=green)
    ans = input(f"Select mode ({rLevel})")
    if ans.isdigit():
        ans = int(ans)
        if 0 <= ans <=3: rLevel = ans
    if rLevel == 0:
        dprint("전체 학생에 대한 자동채점 시작...", log=False, scr=True, color=red)
    if os.access(PLAGIFILE, os.R_OK):
        bCheckPlagi = True
        plagiFr = pd.read_csv(PLAGIFILE, dtype=str, index_col=cSTID, encoding="euc-kr")
else:
    print("채점파일({}) NOT in ({})\nExit...".format(scrFile, HomeDir))
    bHaveAvg = True
    stFolders = []
    # quit(0)

bAllSkip = False
for stf in stFolders:               # Checking Student
    if bAllSkip: continue           # Skip all students
    stName, stId = stf.split('-')
    if skipName:
        if skipName != stName: continue
    skipName = ''
    stId = int(stId.split('_')[0])
    stDir = stdDir+'/'+stf
    pyFiles = os.listdir(stDir)
    print("#"*38)
    if rLevel >= 1:
        ans = input("Scoring for {}({})? (q for Finishing, s for skip to next, 이름 for searching, 0~3 for new mode)".format(stName, stId))
        if len(ans) == 1:
            if ans == "s":
                print(red+"skip to next..."+back)
                continue
            elif ans == "q":
                print(red+"Finishing..."+back)
                bAllSkip = True
                continue
            elif ans.isdigit():
                ans = int(ans)
                if 0 <= ans <=3: rLevel = ans
        elif ans:
            bskipName = False
            for sfn in stFolders:
                if ans in sfn:
                    bskipName = True
                    break
            if bskipName:
                skipName = ans
                continue
    # print()
    bReporting = False
    ntScore = 0
    feedBack = ""
    msgBuffer = []                   # 학생별 상세 채점 피드백
    submitTime = 0                   # 제출시간

    for pNumFr in scrFrame.문제번호.values:
        pNum = int(pNumFr)
        bSubmit = False
        for pyF in pyFiles:
            if pNum == GetPNum(pyF):
                bSubmit = True
                break
        if not bSubmit:
            pyF = "미제출"
        if rLevel >= 2:
            ans = input("문제 채점: {}번 ({}) / s for skip".format(pNum, pyF))
            if ans == 's':
                print("skip to next...")
                continue
        numFrame = scrFrame[scrFrame.문제번호 == pNumFr]

        if len(numFrame) == 0:
            print("No such number({})".format(pNum))
            continue

        # 프로그램 실행 및 비교: 평가파일: ansFile, 정답파일: solFile
        dprint("\n\n{}번 문제:".format(pNum), str(numFrame.제목.values[0]), scr=rLevel>=2)
        dprint("="*VLINE, scr=rLevel>=2)
        dprint("{:*<50s}".format("** 정답해설 프로그램 스크립트 "), scr=rLevel>=2)
        solPgm = numFrame.정답프로그램.values[0]
        mylibs = numFrame.mylib.fillna('').values[0].split('\n')
        myCopy(solPgm.strip(), solFile, src="str", mylib=mylibs)
        dprint(solPgm, scr=rLevel>=2, color=green)
        dprint("="*VLINE, scr=rLevel>=2)

        if not bSubmit:
            feedMsg = "/ 답안 미제출"
            dprint("{:*<50s}".format("** 제출된 프로그램 없음 "), scr=rLevel>=2)
            dprint("="*VLINE, scr=rLevel>=2)
            dprint("{}번 채점결과(0점 / {:.1f}점 만점)  {}".format(pNum, nScrFactor*100, feedMsg), scr=rLevel>=2, color=red)
            if bHaveAvg:
                feedBack += "{}번 (0점 / {:.1f}점 만점, 응시자 평균 {:.1f}) {}\n".format(pNum, nScrFactor*100, scrAvg[pNum-1], feedMsg)
            else:
                feedBack += "{}번({:.1f}점 만점에 0점) {}\n".format(pNum, nScrFactor*100, feedMsg)
            dprint("="*VLINE, '\n', scr=rLevel>=2)
            continue

        pyFile = stDir + '/' + pyF
        dprint("{:*<50s}".format("** 제출된 프로그램 스크립트 "), scr=rLevel>=2)
        ansPgm = myCopy(pyFile, ansFile, mylib=mylibs)
        dprint(ansPgm.strip(), scr=rLevel>=2, color=blue)
        dprint("="*VLINE, scr=rLevel>=2)
        bPlagied = False
        if bCheckPlagi:
            plgs = plagiFr[plagiFr.index == stId]
            plg = plgs[plgs[cPNUM] == str(pNum)]
            if len(plg):
                bPlagied = True
                plagiPgm = plg[cPROGRAM].values[0]
                plagiWho = plg[cWHO].values[0]
                if plagiWho != '정답해설':
                    plagiWho = plagiWho[0] + "**"
                dprint("{:*<50s}".format(f"** 다음 {plagiWho} 프로그램과 90% 이상 일치로 표절 의심 (0점 처리) "), scr=rLevel>=2)
                dprint("{:*<50s}".format("** 표절이 아닌 경우 소명 메일 바람 "), scr=rLevel>=2)
                dprint("{:*<50s}".format(f"** {plagiWho} 프로그램 스크립트 "), scr=rLevel>=2)
                dprint(plagiPgm.strip(), scr=rLevel>=2, color=blue)
                dprint("="*VLINE, scr=rLevel>=2)

        # n개의 Ex에 대해서 처리
        numEx = int(numFrame.nEx.fillna(0))
        if numEx < 1:
            for n in range(1, 100):
                exnIn = f"Ex{n}"
                if exnIn not in numFrame.columns: break
                chkN = numFrame[exnIn].fillna("NA") == "NA"
                if chkN.values[0]: break
            numEx = max(1, n - 1)
        tScr = 0           # 문제의 점수

        # 프로그램 실행 및 비교: 평가파일: ansFile, 정답파일: solFile
        brsBanned = False
        brsDelayed = False
        brsMiss = False
        brsLoop = 0.0
        brsErr = 0.0
        crsDelayed = 0.0    # 지각제출 감점

        # 제출시간 계산 (지각제출 감점율)
        pfTime = os.stat(pyFile).st_mtime
        if pfTime > dTime:
            brsDelayed = True
            crsDelayed = min((pfTime - dTime) / 360000, 0.9)

        # ansFile에 필수어/금지어 포함여부 검사
        crsBanned = 0.0     # 필수/금지어 사용 감점율 (오사용 단어) / (총 단어) * rsBanned
        crsAllWords = 0
        crsWrongWords = 0
        crsOut = 0      # 출력형식 점수
        wdsBanned = numFrame.필수단어.fillna('').values[0].split('\n')
        if wdsBanned == ['']: wdsBanned = []
        bndMissed = []
        bndUsed = []
        ansNCPgm = deComment(ansPgm)

        for wdBan in wdsBanned:
            if wdBan:
                if wdBan not in ansNCPgm:
                    bndMissed.append(wdBan)
                    brsBanned = True;
        crsAllWords += len(wdsBanned)
        crsWrongWords += len(bndMissed)
        wdsBanned = numFrame.금지단어.fillna('').values[0].split('\n')
        if wdsBanned == ['']: wdsBanned = []
        for wdBan in wdsBanned:
            if wdBan:
                if wdBan in ansNCPgm:
                    bndUsed.append(wdBan)
                    brsBanned = True;
        crsAllWords += len(wdsBanned)
        crsWrongWords += len(bndUsed)
        if crsWrongWords: crsBanned = crsWrongWords / crsAllWords * rsBanned

        if crsBanned:
            dprint(f"- 필수기능/금지기능 오사용 ({crsBanned:.0%} 감점)", scr=rLevel>=2)
            if bndMissed:
                dprint(f"- 필수기능 {bndMissed} 사용 안됨", scr=rLevel>=2)
            if bndUsed:
                dprint(f"- 금지기능 {bndUsed} 사용 됨", scr=rLevel>=2)
        dprint("="*VLINE, scr=rLevel>=2)

        # 각 예시입력 처리
        exeMode = numFrame.채점모드.fillna('char').values[0]   # manual(프로그램), serial, user(실행결과), word, byte, char
        if exeMode == '': exeMode = "serial"
        userFeedback = False
        userfbMsg = ''
        if exeMode == 'manual':
            for en in range(1, numEx+1):   # 예시 입력에 대해
                exScr = 0
                ex = 'Ex'+str(en)
                exTest = str(numFrame[ex].values[0])  # Test Data
                myCopy(exTest, inFile, src="str", type="text")
                # Test Case
                dprint(f"Test Case {en}:", color=red, scr=rLevel>=3 or exeMode=="manual")
                dprint("="*VLINE, scr=rLevel>=3 or exeMode=="manual")
                dprint("{:~<50s}\n{}".format("~~ 입력 데이터 ~~", exTest), color=green, scr=rLevel>=3 or exeMode=="manual")
                dprint("{:~<50s}".format("~~ 채점 ~~"), color=green, scr=rLevel>=3 or exeMode=="manual")
                #run = os.system(f"python.exe -m idlelib {ansFile}")
                #run = os.system(f"C:/Users/ypjun/AppData/Local/Programs/Python/Python39/pythonw.exe -m idlelib {ansFile}")
                # run = os.system("aaa.bat")
                sScr = "No digit"
                while not sScr.isdigit(): # pyautogui.hotclick('f1'), ('ctrl','shift','x',interval=0.2)
                    pg.hotkey('f1')
                    sScr = input("Enter Score by Manual {}-{}번 (100점): ".format(pNum, ex))
                    if sScr == '': sScr = '100'
                exScr = int(sScr)
                enfbMsg = ''
                if exScr < 100:
                    userFeedback = True
                    enfbMsg = input("감점 사유: ")
                    if enfbMsg:
                        if userfbMsg: userfbMsg += ', ' + enfbMsg
                        else: userfbMsg = enfbMsg
                        enfbMsg = '('+enfbMsg+')'
                    scrUser = (100 - exScr) / numEx
                tScr += exScr / numExal
                dprint(f"Test Case {en} 득점률: {round(exScr)}% {enfbMsg}", scr=rLevel>=3)
                dprint("="*VLINE, scr=rLevel>=3)
        else:
            mustRatioAvg = 0
            for en in range(1, numEx+1):   # 예시 입력에 대해
                exScr = 0
                ex = 'Ex'+str(en)
                exTest = str(numFrame[ex].values[0])  # Test Data
                solCode, solOut, errOut = execPython(solFile, exTest)
                if solCode != SUCCESS:
                    print("정답프로그램 오류 ({}) in {}".format(solCode, solFile))
                    print("문제: {}##{}##{}##{}".format(stf, pyF, ex, exTest))
                    input("Skip to next...")
                    continue
                ansCode, ansOut, errOut = execPython(ansFile, exTest)
                # Answer 답 길이가 너무 길지 않도록
                # axLen = min(1000, len(ansOut))
                # ansOut = ansOut[:maxLen]
                solOut = ml_rstrip(solOut)
                ansOut = ml_rstrip(ansOut)

                # Test Case
                dprint(f"Test Case {en}:", color=red, scr=rLevel>=3 or exeMode=="user")
                dprint("="*VLINE, scr=rLevel>=3 or exeMode=="user")
                nsolOut = numbering(solOut, "S")
                nansOut = numbering(ansOut, "A")
                dprint("{:~<50s}\n{}".format("~~ 입력 데이터 ~~", exTest), color=green, scr=rLevel>=3 or exeMode=="user")
                dprint("{:~<50s}\n{}".format("~~ 정답해설 프로그램 실행 결과 ~~", nsolOut), color=green, scr=rLevel>=3 or exeMode=="user", end='')
                dprint("{:~<50s}\n{}".format("~~ 제출된 프로그램 실행 결과 ~~", nansOut), color=green, scr=rLevel>=3 or exeMode=="user", end='')

                # 정답률: 필수 정답 포함 검사 (srANS)
                scrTmpSim = 0
                exOUT = str(numFrame[ex+'-O'].fillna('').values[0])
                if exOUT != '':
                    exOUT = exOUT.split('\n')
                else: exOUT = []
                exN = len(exOUT)
                if exN == 0: scrTmpMust = srANS
                else: scrTmpMust = 0
                lstMust = []
                ansSearch = ansOut
                solOutSR = solOut
                ansOutSR = ansOut
                for exO in exOUT:
                    exOn = exO.split('@@@')
                    if len(exOn) == 2:
                        exOs = exOn[1]
                    else:
                        exOs = exOn[0]
                    reSch = re.search(exOs, ansSearch)
                    if reSch:  # != None
                        scrTmpMust += srANS / exN
                        if exeMode == 'serial':
                            schN = reSch.end()
                            ansSearch = ansSearch[schN:]
                        elif exeMode == 'onetime':
                            ansSearch = ansSearch[:reSch.start()] + ansSearch[reSch.end():]
                            # 필수단어 제거
                            reSearch = re.search(exOs, solOutSR)
                            if reSearch: solOutSR = solOutSR[:reSearch.start()] + solOutSR[reSearch.end():]
                            reSearch = re.search(exOs, ansOutSR)
                            if reSearch: ansOutSR = ansOutSR[:reSearch.start()] + ansOutSR[reSearch.end():]
                    else:
                        brsMiss = True
                        lstMust.append(exOn[0])

                # 유사도 검사 (srSIM)
                SimRatio = SimRate(solOutSR, ansOutSR, unit=exeMode)
                if SimRatio < 100:
                    lnsolOut = nsolOut.splitlines()
                    dnsolOut = ''.join(solOut.split(' ')).splitlines()
                    lnansOut = nansOut.splitlines()
                    dnansOut = ''.join(ansOut.split(' ')).splitlines()
                    naLines = (len(dnsolOut) + len(dnansOut))
                    neLines = 0
                    diffs = list(dl.ndiff(dnsolOut, dnansOut))
                    dprint("{:~<50s}".format("~~ 실행 결과 불일치 내용 ~~"), color=green, scr=rLevel>=3 or exeMode=="user")
                    for dfLine in diffs:
                        if len(dfLine) < 4: continue
                        if dfLine[0] == '-':     # Solution에만 있음
                            dnum = dnsolOut.index(dfLine[2:])
                            dprint("▶ {}".format(lnsolOut[dnum]), scr=rLevel>=3 or exeMode=="user")
                            neLines -= 1
                        elif dfLine[0] == '+':     # Answer에만 있음
                            dnum = dnansOut.index(dfLine[2:])
                            dprint("▷ {}".format(lnansOut[dnum]), scr=rLevel>=3 or exeMode=="user")
                            neLines += 1
                    NLRatio = 100 - (100 * abs(neLines) / naLines)
                else: NLRatio = 100
                SimRatio *= (NLRatio * srNL / 100)

                dprint("{:~<50s}".format("~~ 채점 ~~"), color=green, scr=rLevel>=3 or exeMode=="user")
                # 결과 출력 (정답률, 정확도)
                if exeMode == "user":
                    exaRatio = SimRatio * (srSIM / srSIM)
                    userScr = input("Enter user score ({:.0f}): ".format(exaRatio))
                    if userScr.isdigit():
                        exaRatio = int(userScr)
                        if exaRatio < 100: userFeedback = True
                        scrUser = (100 - exaRatio) / numEx
                    scrTmpMust = exaRatio / 2
                    SimRatio = exaRatio
                else:
                    if lstMust and exOUT:
                        mustRatio = (len(exOUT) - len(lstMust)) / len(exOUT)
                        dprint("정답률 ({:.0%}) : {} 미 도출".format(mustRatio, lstMust), scr=rLevel>=3)
                    else:
                        dprint("정답률 (100%)", scr=rLevel>=3)
                        mustRatio = 1
                    mustRatioAvg += mustRatio / numEx
                dprint("정확도 ({:.0f}%)".format(SimRatio), scr=rLevel>=3 or exeMode=="user")
                scrTmpSim = SimRatio * (srSIM / 100)
                exScr = scrTmpMust + scrTmpSim

                # 문제-테스트케이스 점수 합산
                exFScr = exScr / numEx
                crsOut += exFScr
                if ansCode == INFINITE_LOOP:
                    exFScr *= rsLoop
                    brsLoop += 1 / numEx
                    dprint("무한반복 오류 10% 감점", scr=rLevel>=3)
                elif ansCode == EXE_ERROR:
                    exFScr *= rsErr
                    brsErr += 1 / numEx
                    dprint("실행오류(10% 감점): ", errOut, scr=rLevel>=3)

                tScr += exFScr
                dprint(f"Test Case {en} 득점률: {round(exFScr * numEx)}%", scr=rLevel>=3)
                dprint("="*VLINE, scr=rLevel>=3)
        if brsBanned:   # 금지어 사용 / 필수어 미사용 감점 적용
            tScr *= (1.0 - crsBanned)
        if tScr < 100: brsStyle = True
        else: brsStyle = False
        if brsDelayed:   # 지각제출 감점 적용
            tScr *= (1.0 - crsDelayed)
        feedMsg = ''
        nScrFactor = scrFactor[pNum-1]
        nScore = tScr * nScrFactor
        if userFeedback:
            if userfbMsg == '':
                userfbMsg = "출력형식 미흡"
            nuserfbMsg = input("{}번({:.0f}점) 감점요인: ".format(pNum, tScr))
            if nuserfbMsg != '':
                userfbMsg = nuserfbMsg
            feedMsg += " / {} ({:.1f}점 감점)".format(userfbMsg, scrUser*nScrFactor)
            brsStyle = False
            bReporting = True
        if brsStyle:
            scrStyle = (100-crsOut-(1-mustRatioAvg)*50)*nScrFactor
            if scrStyle > 0:
                feedMsg += " / 출력형식 불일치 ({:.1f}점 감점)".format(scrStyle)
                bReporting = True
            if brsMiss: feedMsg += " / 정답도출 실패 ({:.1f}점 감점)".format((1-mustRatioAvg)*50*nScrFactor)
        if brsErr: feedMsg += " / 실행오류 ({:.1%} 추가 감점)".format(brsErr * (1 - rsErr))
        if brsLoop: feedMsg += " / 무한반복 오류 ({:.1%} 추가 감점)".format(brsLoop * (1 - rsErr))
        if brsBanned: feedMsg += " / 금지기능/필수기능 오사용 ({:.0%} 추가 감점)".format(crsBanned)
        if brsDelayed: feedMsg += " / {:.1f}시간 지각제출 ({:.0%} 추가 감점)".format(crsDelayed * 100, crsDelayed)
        dprint("{}번 채점결과 ({:.1f}% => 환산점수: {:.1f}점 / {:.1f}점 만점)".format(pNum, tScr, nScore, nScrFactor*100), scr=rLevel>=2, color=red)
        if feedMsg:
            dprint("감점요인 {}".format(feedMsg), scr=rLevel>=2, color=red)
        if bPlagied:
            nScore = 0
            feedMsg = "표절 의심 0점 처리"
            dprint("====> 표절 의심으로 0점 처리 (표절이 아닌 경우 소명 메일 보내기 바람)", scr=rLevel>=2, color=red)
        if rLevel >= 2:
            nfeed = input("추가 피드백: ")
            if nfeed != '':
                feedMsg += " / " + nfeed
        if bHaveAvg:
            feedBack += "{}번 ({:.1f}점 / {:.1f}점 만점, 응시자 평균 {:.1f}) {}\n".format(pNum, nScore, nScrFactor*100, scrAvg[pNum-1], feedMsg)
        else:
            feedBack += "{}번({:.1f}점 만점에 {:.1f}점) {}\n".format(pNum, nScrFactor*100, nScore, feedMsg)
            scrSum[pNum-1] += nScore
        ntScore += nScore
        dprint("="*VLINE, '\n', scr=rLevel>=2)
        if brsBanned or brsDelayed or brsMiss or brsLoop or brsErr: bReporting = True
    print()
    ntScore = min(100.0, ntScore+0.1)
    feedBack += "총점: {:.0f}점".format(ntScore)
    if ntScore < MINSCORE:
        feedBack += "=> {:.0f}점 / 제출 최소점수 적용".format(MINSCORE)
        ntScore = MINSCORE
    if bHaveAvg:
        for nth in range(scrNumSt):
            if scrScores[nth] <= round(ntScore,1): break
        nth += 1
        feedBack += " (응시자 {}명 중 {}등, 평균 {:.0f}점)".format(scrNumSt, nth, sum(scrAvg))
    else: scrScores.append(round(ntScore,1))
    print(green+f"{stName}({stId})\n"+red+feedBack+back)
    ## 개별 채점 보고서 생성: 채점결과/채점결과_이름_학번, if(bReporting):  
    scFolder = "./채점결과"
    if not os.access(scFolder, os.X_OK): os.mkdir(scFolder)
    scrReport = scFolder + "/Scoring_{}_{}.txt".format(stName, stId)
    fScr = open(scrReport, "wt", encoding='utf-8')
    fScr.write("{}\n".format(hw_name))
    fScr.write("채점 요약 보고서 (이름: {} / 학번: {})\n".format(stName, stId))
    fScr.write("#" * 38 + '\n')
    fScr.write(feedBack)
    fScr.write("\n\n{:=^60s}\n\n".format(" 세부 채점 내용 "))
    for msg in msgBuffer:
        fScr.write(msg+'\n')
    fScr.close()

dprint("\n\n전체 학생에 대한 자동채점 종료...", log=False, scr=True, color=blue)
if not bHaveAvg:
    print("==== 문제별 응시자 평균 점수 ====")
    for i in range(len(scrSum)):
        scrAvg[i] = str(round(scrSum[i] / scrNumSt, 1))
        print("{}번 평균: {}".format(i+1, scrAvg[i]))
    print("전체 평균: {}".format(round(sum(scrSum) / scrNumSt), 1))
    scrScores.sort(reverse=True)
    scrScores = list(str(n) for n in scrScores)
    ct = ' '.join(scrAvg) + '\n' + ' '.join(scrScores)
    with open(avgScrFile, "wt") as fp:
        fp.write(ct)
os.chdir(cdir)

자동채점(Sample) 실행 모드 
  0 : 전체 자동화 (전체 학생에 대하여 한번에 채점) 
  1 : 학생별 자동화 (학생별로 구분하여 채점) 
  2 : 문제별 자동화 (학생에 대해 문제별로 구분하여 채점) 
  3 : 예시별 자동화 (각 문제에 대한 테스트 입력별로 구분하여 채점) 
Select mode (0)
전체 학생에 대한 자동채점 시작... 
######################################

강민송(2021240526)
1번 (10.0점 / 100.0점 만점, 응시자 평균 8.3)  / 90.0시간 지각제출 (90% 추가 감점)
총점: 10점 (응시자 1명 중 1등, 평균 44점)


전체 학생에 대한 자동채점 종료... 


In [6]:
plagiInfo.pnum

stid
2022240592    1
2022240592    4
Name: pnum, dtype: object

In [9]:
mylibs

['random', 'time']

In [4]:
(pfTime - dTime) / 360000

-0.7181388888888889

In [7]:
crsDelayed

0.0

In [24]:
 t.ctime(dTime)

'Tue Jun 21 10:54:47 2022'

In [25]:
 t.ctime(pfTime)

'Thu Jun 16 12:26:10 2022'

In [57]:
# 파일 시간
import time as t
pFiles = os.listdir()
pF = pFiles[0]
ftSec = os.stat(pF).st_mtime
ascTm = t.ctime(ftSec)
# t.ctime(os.stat(pF).st_atime)
print(ftSec, ascTm)

1646128894.8581963 Tue Mar  1 19:01:34 2022


In [47]:
t.ctime(os.stat(pF).st_ctime)

'Tue Mar  1 16:15:03 2022'

In [42]:
# 년월일 => ctime
os.stat(pF)

os.stat_result(st_mode=33206, st_ino=24206847997181892, st_dev=3735091627, st_nlink=1, st_uid=0, st_gid=0, st_size=240, st_atime=1646128894, st_mtime=1646128894, st_ctime=1646118903)

In [15]:
delayTime = input("지각시간 (Y-M-D-H-M)").split('-')
if len(delayTime) != 5: dTime = 0
else:
    dt = [int(n) for n in delayTime]
    dTime = t.mktime((dt[0], dt[1], dt[2], dt[3], dt[4], 0, 1, 60, 0))

지각시간 (Y-M-D-H-M)2022-3-1-10-0


In [53]:
os.listdir('응시자/권용은-2021286012_5883892_assignsubmission_file_')

['권용은 1번.py', '권용은 2번.py', '권용은 3번.py', '권용은 4번.py', '권용은 5번.py']

In [27]:
t.ctime(dTime)

'Wed Dec 15 14:00:00 2021'

In [13]:
delayTime

'2022-04-15 14-01'

In [55]:
os.stat(pyFile)

os.stat_result(st_mode=33206, st_ino=5348024557767885, st_dev=3735091627, st_nlink=1, st_uid=0, st_gid=0, st_size=240, st_atime=1644809653, st_mtime=1639531934, st_ctime=1644809653)

In [56]:
t.ctime(os.stat(pyFile).st_mtime)

'Wed Dec 15 10:32:14 2021'

In [54]:
pyFile

'C:/Users/YPJ/Google 드라이브(ypjun7@gmail.com)/JupyterWork/GradeAnal/2022-01(성적)/기말고사채점/CT01/응시자/문서영-2021286032_5883886_assignsubmission_file_/문서영(4).py'